## Load the extension

In [1]:
%load_ext autoreload
%aimport -sparkmagic # it loses the references to the sessions if it reloads
%autoreload 2

In [2]:
%load_ext livy_uploads.magics

## Fetching remote variable

In [3]:
print(list(sorted(globals())))

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,None,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['HiveContext', 'StreamingContext', '__builtins__', 'cloudpickle', 'sc', 'spark', 'sqlContext']

In [4]:
%local

print(list(sorted(globals())))

['In', 'Out', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__session__', '__spec__', '_dh', '_i', '_i1', '_i2', '_i3', '_i4', '_ih', '_ii', '_iii', '_oh', 'display_dataframe', 'exit', 'get_ipython', 'ip', 'open', 'quit']


In [5]:
from datetime import datetime

now = datetime.now().astimezone()
now

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

datetime.datetime(2025, 12, 20, 21, 2, 22, 103000, tzinfo=datetime.timezone(datetime.timedelta(0), 'UTC'))

In [6]:
%get_obj_from_spark -n now

In [7]:
%local

now

datetime.datetime(2025, 12, 20, 21, 2, 22, 103000, tzinfo=datetime.timezone(datetime.timedelta(0), 'UTC'))

In [8]:
%local

from datetime import datetime

delta = (datetime.now().astimezone() - now)
assert delta.total_seconds() < 30

## Sending local variable

In [9]:
%local

foo = {2, 3, 4}


In [10]:
%send_obj_to_spark -n foo

In [11]:
foo_total = sum(foo)

assert foo == {2, 3, 4}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
%get_obj_from_spark -n foo_total

In [13]:
%local

assert foo_total == 9

## Running commands

In [14]:
%%shell_command

ls -lahF .

total 96K
drwxrwxrwt  1 root root 4.0K Dec 20 21:02 ./
drwxr-xr-x  1 root root 4.0K Dec 20 21:01 ../
-rw-------  1 app  app   22K Dec 20 21:02 4818623367592642843
drwxr-xr-x  2 app  app  4.0K Dec 20 21:02 blockmgr-d98206a4-d920-4f49-b294-9378ee6c5d5b/
drwxr-xr-x  2 app  app  4.0K Dec 20 21:02 hsperfdata_app/
drwxr-xr-x  2 root root 4.0K Oct 19  2024 hsperfdata_root/
-rw-------  1 app  app  3.9K Dec 20 21:02 livyConf3890629206090919141.properties
-rw-------  1 app  app  8.4K Dec 20 21:01 magics.ipynb
drwx------  2 app  app  4.0K Dec 20 21:02 rsc-tmp4569829120044733179/
drwx------  3 app  app  4.0K Jan  9  2025 sample-dir/
drwxr-xr-x  2 app  app  4.0K Dec 20 20:18 spark/
drwx------ 13 app  app  4.0K Dec 20 21:02 spark1650519457699558222/
drwx------  4 app  app  4.0K Dec 20 21:02 spark-432eb187-770a-4346-9cef-e53de08657ca/
drwx------ 13 app  app  4.0K Dec 20 21:01 spark7950148411163509628/
drwxr-xr-x  2 app  app  4.0K Dec 20 21:02 spark-ae56d3e8-f839-4291-817b-44800913c2ec/
drwxr-xr-x  2 

In [15]:
%%shell_command

bash -c 'echo foo && exit 42'

foo
$ command exited with code 42

In [16]:
%%local

assert shell_output == 'foo\n'
assert shell_returncode == 42


## Sending local file

In [17]:
%local !ls -lahF

total 100K
drwxrwxr-x  5 app app 4.0K Dec 20 21:02 ./
drwxrwxr-x 20 app app 4.0K Dec 20 19:59 ../
drwxr-xr-x  2 app app 4.0K Jan 13  2025 .ipynb_checkpoints/
-rw-rw-r--  1 app app  42K Dec 20 21:02 magics.ipynb
drwxrwxr-x  3 app app 4.0K Jan  9  2025 sample-dir/
drwxrwxr-x  2 app app 4.0K Dec 20 19:45 spark/
-rw-rw-r--  1 app app  33K Dec 20 19:45 test-spark-another-version.ipynb


In [18]:
%send_path_to_spark -p magics.ipynb

Uploaded magics.ipynb to /tmp/magics.ipynb

In [19]:
%%shell_command

ls -lahF | grep magics

-rw-------  1 app  app   42K Dec 20 21:02 magics.ipynb
$ command exited with code 0

In [20]:
%%local

assert 'magics.ipynb' in shell_output

## Sending local directory

In [21]:
%local !find sample-dir/

sample-dir/
sample-dir/inner
sample-dir/inner/bar.txt
sample-dir/foo.txt


In [22]:
%send_path_to_spark -p sample-dir/

Uploaded sample-dir to /tmp/sample-dir

In [23]:
%%shell_command
pwd

/tmp
$ command exited with code 0

In [24]:
%%shell_command

find "$PWD/sample-dir"

/tmp/sample-dir
/tmp/sample-dir/inner
/tmp/sample-dir/inner/bar.txt
/tmp/sample-dir/foo.txt
$ command exited with code 0

In [25]:
%%local

assert 'sample-dir/' in shell_output

## Following session logs

In [26]:
%logs_follow -p 500

stdout: 
25/12/20 21:02:12 INFO RSCDriver: Received job request 25181c0f-c239-473d-bed2-87b06a984908
25/12/20 21:02:12 INFO RSCDriver: SparkContext not yet up, queueing job request.
25/12/20 21:02:14 INFO SparkEntries: Starting Spark context...
25/12/20 21:02:14 INFO SparkContext: Running Spark version 3.3.2
25/12/20 21:02:14 INFO ResourceUtils: ==============================================================
25/12/20 21:02:14 INFO ResourceUtils: No custom resources configured for spark.driver.
25/12/20 21:02:14 INFO ResourceUtils: ==============================================================
25/12/20 21:02:14 INFO SparkContext: Submitted application: livy-session-1
25/12/20 21:02:14 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1000, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
25/12/2

In [27]:
%logs_follow -p 500

No new logs

In [28]:
sc._gateway.jvm.java.lang.System.err.println('Hello World')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
%logs_follow -p 500

Hello World


In [30]:
%local

assert 'Hello World' in '\n'.join(logs_lines)